In [1]:
import pandas as pd
import pingouin as pg
import numpy as np
from itertools import product

In [2]:

factors = ['FatorA' ,'FatorB', 'FatorC']
levels_list = [['low','high'],['L','H'],['-','+','++']]
replicates = 5
total = np.prod([len(x) for x in levels_list]) * replicates
y_measures = np.random.rand(total) #([28,25,27,18,19,23,36,32,32,31,30,29,28,25,22,18,19,23,12,32,40,31,30,29])

In [3]:
def generate_dataframe(measures, factors, levels_list, replicates):
    
    lines = []
    for factor_combination in product(*levels_list):
        line = {}
        for idx, factor in enumerate(factors):
            line[factor] = factor_combination[idx]
        for k in range(replicates):
            lines.append(line)
    df = pd.DataFrame(lines,columns=factors)
    df['y'] = measures
    #df['trial'] = list(range(replicates)) * int(len(measures) / replicates)
    return df

In [4]:
df = generate_dataframe(y_measures, factors, levels_list, replicates)

In [6]:
df.head()

,FatorA,FatorB,FatorC,y
0,low,L,-,0.016975
1,low,L,-,0.813731
2,low,L,-,0.540931
3,low,L,-,0.127740
4,low,L,-,0.110234


In [7]:
df.anova(dv='y', between=factors, ss_type=3).round(3)


,Source,SS,DF,MS,F,p-unc,np2
0,FatorA,0.253,1.0,0.253,3.111,0.084,0.061
1,FatorB,0.140,1.0,0.140,1.722,0.196,0.035
2,FatorC,0.000,2.0,0.000,0.001,0.999,0.000
3,FatorA * FatorB,0.021,1.0,0.021,0.263,0.611,0.005
4,FatorA * FatorC,0.271,2.0,0.136,1.664,0.200,0.065
5,FatorB * FatorC,0.479,2.0,0.239,2.940,0.062,0.109
6,FatorA * FatorB * FatorC,0.020,2.0,0.010,0.120,0.887,0.005
7,Residual,3.909,48.0,0.081,NaN,NaN,NaN
